<a href="https://colab.research.google.com/github/ibrahimkaya754/GAN/blob/main/dcgan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Setup

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
cd drive/MyDrive/ESEN/Scripts/CGAN_with_Classifier

In [ ]:
## MODULES AND THE DATA IMPORT
from numpy import expand_dims
from numpy import zeros
from numpy import ones
from numpy.random import randn
from numpy.random import randint
from keras.datasets.fashion_mnist import load_data
from tensorflow.keras.optimizers import *
from tensorflow.keras.models import Model
from tensorflow.keras.layers import *
from tensorflow.keras.initializers import *
import tensorflow as tf
import numpy as np
import pickle
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from numpy import asarray
from numpy.random import randn
from numpy.random import randint
from keras.models import load_model
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.constraints import Constraint
from tensorflow.keras.losses import *
#from MoGeNA import *
from sklearn.preprocessing import *
from tensorflow.python.platform import build_info as tf_build_info
from scipy import spatial
import os
print(tf_build_info.build_info)

In [ ]:
os.getcwd()

In [ ]:
gpu_devices = tf.config.experimental.list_physical_devices('GPU')
for device in gpu_devices:
    tf.config.experimental.set_memory_growth(device, True)
    print("tensorflow version is: ",tf.__version__)

In [ ]:
# Load The Data
def load_wifi_data(dataDict):
  for key in dataDict.keys():
    print("Preparing %s data" % (key))
    data_wifi   = pickle.load(open(dataDict[key]["datalocation"],"rb"))
    labels_wifi = pickle.load(open(dataDict[key]["labellocation"],"rb"))
    
    labels  = []
    
    print("data_shape for %s is %s: " % (key,data_wifi.shape)) 
    print('label shape for %s is %s: ' % (key,labels_wifi.shape))

    for lbl in labels_wifi:
        if lbl not in labels:
            labels.append(lbl)

    print("labels: ",labels)
    dataDict[key]['data']   = data_wifi
    dataDict[key]['labels'] = labels_wifi

    dumbarray = np.zeros((dataDict[key]['labels'].shape[0],16),dtype=np.int16)
    for ii in range(dataDict[key]['labels'].shape[0]):
        indice = labels_wifi[ii]
        dumbarray[ii,indice] = 1

    dataDict[key]['label1hot']       = dumbarray
    dataDict[key]['numberOfClasses'] = len(labels)
    print("number of classes: ",dataDict[key]['numberOfClasses'])

    dataDict[key]["data"]= dataDict[key]['data'].reshape(dataDict[key]['data'].shape[0],
                                                         dataDict[key]['data'].shape[1],
                                                         dataDict[key]['data'].shape[2],1)
    print('dataset shape: ', dataDict[key]["data"].shape)
    print('max value before scaling: ', np.max(dataDict[key]["data"]))
    print('min value before scaling: ', np.min(dataDict[key]["data"]))
    print("*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--")
  return dataDict

In [ ]:
data={"TrainData"      :{"datalocation"  : "../newDATA/trainingData_13042022 15:56:46",
                         "labellocation" : "../newDATA/trainingLabel_13042022 15:56:46"},
      "ValidationData" :{"datalocation"  : "../newDATA/knowntestData_13042022 15:56:46",
                         "labellocation" : "../newDATA/knowntestLabel_13042022 15:56:46"},
      "UnknownData"    :{"datalocation"  : "../newDATA/unknowntestData_18042022 15:36:59",
                         "labellocation" : "../newDATA/unknowntestLabel_18042022 15:36:59"}}

In [ ]:
data = load_wifi_data(data)

In [ ]:
def scaling(dataDict,scalingA2="TrainData"):

  scalerx0 = MinMaxScaler((-0.5,0.5))
  scalerx1 = MinMaxScaler((-0.5,0.5))

  scx0 = scalerx0.fit(dataDict[scalingA2]["data"][:,:,0,0])
  scx1 = scalerx1.fit(dataDict[scalingA2]["data"][:,:,1,0])

  for key in dataDict.keys():
    dataDict[key]["dataScaled"] = dataDict[key]["data"]
    dataDict[key]["dataScaled"][:,:,0,0]= scx0.transform(dataDict[key]["data"][:,:,0,0])
    dataDict[key]["dataScaled"][:,:,1,0]= scx1.transform(dataDict[key]["data"][:,:,1,0])
    print('min, max value of I after scaling %s dataset: %.3f, %.3f' % (key,np.min(dataDict[key]["dataScaled"][:,:,0,0]),
                                                                              np.max(dataDict[key]["dataScaled"][:,:,0,0])))
    print('min, max value of Q after scaling %s dataset: %.3f, %.3f' % (key,np.min(dataDict[key]["dataScaled"][:,:,1,0]),
                                                                              np.max(dataDict[key]["dataScaled"][:,:,1,0])))

  
  return dataDict,[scx0,scx1]
data,[scx0,scx1] = scaling(data)

In [ ]:
def L2normalize(arg):
  import keras.backend as K
  arg = K.l2_normalize(arg,axis=1)
  return arg

In [ ]:
class model():
  def __init__(self, dataDict, AppendOptimizedParams= True,
               location= "../CGAN_with_Classifier_Models/",
               in_shape=(1024,2,1), loadSavedModel= True,
               filters=[64,32,16,8], neurons=50, n_classes=16,
               strides=[(2,1),(2,2),(2,2),(2,2)], batchSize = 64,
               alpha= 0.2, kernelSize= 3, Drop= 0.2, epochs = 50,
               learningRate= 0.001, name="Module1",
               best_params= [211.0776,207.0222,235.648,
                             129.7916,232.7917,0.3253,
                             6.730,0.237,0.0002942,1024,5]):
    self.in_shape       = in_shape
    self.n_classes      = n_classes
    self.filters        = filters
    self.neurons        = neurons
    self.strides        = strides
    self.alpha          = alpha
    self.kernelSize     = kernelSize
    self.Drop           = Drop
    self.learningRate   = learningRate
    self.modelName      = name
    self.data           = dataDict
    self.batchSize      = batchSize
    self.epochs         = epochs
    self.location       = location
    self.loadSavedModel = loadSavedModel
    self.bestParams     = best_params
    self.AppendOptimizedParams = AppendOptimizedParams
    self.models         = {}

    if self.AppendOptimizedParams:
      self.__AppendOptimumPArams()
    
    self.CallBack()
    self.define_classifier()
    self.createSiameseModel()

  def define_classifier(self):
    self.opt = Adam(lr=self.learningRate)
    in_image = Input(shape=self.in_shape)
    layer1   = Reshape((64,32,1))(in_image)
    
    for fltr,strd in zip(self.filters,self.strides):
        layer1 = Conv2D(int(round(fltr)), (int(round(self.kernelSize)),int(round(self.kernelSize))), 
                        strides=strd, padding='same')(layer1)
        layer1 = LeakyReLU(alpha=self.alpha)(layer1)

    flatten                 = Flatten()(layer1)
    layer1                  = Dropout(self.Drop)(flatten)
    featuredescriptor       = Dense(int(round(self.neurons)))(layer1)
    #featuredescriptor      = Lambda(L2normalize)(featuredescriptor)
    layer1                  = LeakyReLU(alpha=self.alpha)(featuredescriptor)
    layer1                  = Dropout(self.Drop)(layer1)
    out                     = Dense(16,activation='softmax')(layer1)

    self.classifier  = Model(in_image, out)
    
    if self.loadSavedModel:
      self.classifier.load_weights(self.location+self.modelName+".hdf5")
    
    print("NETWORK SUMMARY")
    print("--------------------------------------")
    self.classifier.summary()

    tf.keras.utils.plot_model(self.classifier,show_shapes=True)

    self.siam1 = Model(in_image,featuredescriptor)
    self.siam2 = Model(in_image,featuredescriptor)

    self.models[self.modelName]= {"ClassifierModel": self.classifier,
                                  "Location"       : self.location+self.modelName+".hdf5"}

  def trainClassifier(self,isTrain=True,epochs=50,batchSize=1024):
    self.epochs    = epochs
    self.batchSize = batchSize
    self.classifier.compile(loss='categorical_crossentropy', optimizer=self.opt, metrics=['accuracy'])
    if isTrain:
      self.history = self.models[self.modelName]["ClassifierModel"].fit(data['TrainData']["dataScaled"], 
                                                                        data['TrainData']["label1hot"], 
                                                                        batch_size=int(round(self.batchSize)), 
                                                                        epochs=int(round(self.epochs)), 
                                                                        validation_data=(data['ValidationData']["dataScaled"],
                                                                                         data['ValidationData']["label1hot"]), 
                                                                        verbose=1, shuffle=True,
                                                                        callbacks=[self.callbacks["checkpoint"], 
                                                                                   self.callbacks["reduce_lr"]])
      self.models[self.modelName]["modelhistory"] = self.history 
                                    
  def CallBack(self):
    self.callbacks = {}
    self.callbacks["checkpoint"]     =  ModelCheckpoint(filepath=self.location+self.modelName+".hdf5", 
                                        monitor='val_accuracy', verbose=1, 
                                        save_best_only=True, period=1, 
                                        mode='max',save_weights_only=False)
    self.callbacks["reduce_lr"]      =  ReduceLROnPlateau(monitor='val_accuracy', 
                                        factor=0.98,
                                        patience=10, 
                                        min_lr=0.0000001, mode='max', verbose=1)
    self.callbacks["early_stopping"] =  EarlyStopping(monitor='val_accuracy', 
                                                      patience=250, verbose=1, 
                                                      mode='auto')
  
  ## __AppendOptimumPArams appends the optimum parameters obtained in GA to network                                                     
  def __AppendOptimumPArams(self):
    ## This function appends the optimum values 
    ## obtained from GA to the classifier model.
    self.filters      = self.bestParams[0:4]
    self.neurons      = self.bestParams[4]
    self.alpha        = self.bestParams[5]
    self.kernelSize   = self.bestParams[6]
    self.Drop         = self.bestParams[7]
    self.learningRate = self.bestParams[8]
    self.batchSize    = int(round(self.bestParams[9]))
    self.epochs       = int(round(self.bestParams[10]))
    print("\nOPTIMUM PARAMETER SET APPENDED")
    print("--------------------------------------")
 
  ## __distance just calculates the element-wise absolute difference of two tensors
  def __distance(self,args):
    tensor1,tensor2 = args
    dist = tf.abs((tensor1 - tensor2))
    return dist
  
  def createSiameseModel(self):
    inp1     = Input(shape=(1024,2,1),name="Siaminput1")
    inp2     = Input(shape=(1024,2,1),name="Siaminput2")
    d1out    = self.siam1(inp1)
    d2out    = self.siam2(inp2)
    l1dist   = Lambda(self.__distance)([d1out,d2out])
    l1dist   = Lambda(L2normalize)(l1dist)
    out      = Dense(1,activation="sigmoid")(l1dist)
    self.Siam = Model([inp1,inp2],out)
    self.Siam.compile(loss='binary_crossentropy', optimizer=self.opt, metrics=['accuracy'])
    
    print("\n\nSIAM NETWORK SUMMARY")
    print("--------------------------------------")
    self.Siam.summary()
    self.models[self.modelName]["SiamModel"] = self.Siam
  
  ## __createSiamDataCouples creates data couples for siamese network training
  def __createSiamDataCouples(self,batchSize,dataset):
    labels_siamese = list(np.ones(batchSize))
    siamese_inp1   = []
    siamese_inp2   = []
    indiceCouples  = np.random.randint(0,len(dataset["dataScaled"]),(batchSize,2))
    for ii in range(batchSize):
      indice1 = indiceCouples[ii,0]
      indice2 = indiceCouples[ii,1]
      siamese_inp1.append(dataset["dataScaled"][indice1])
      siamese_inp2.append(dataset["dataScaled"][indice2])
      if dataset["labels"][indice1]==dataset["labels"][indice2]:
        labels_siamese[ii] = 0.0
    return np.array(siamese_inp1),np.array(siamese_inp2),np.array(labels_siamese)

  def trainSiamese(self, Nepochs=100, batchSize=512,
                   trainFeatureExtractors= False, 
                   load_saved_data=False):   
      
    batchPerEp    = int(self.data["TrainData"]["dataScaled"].shape[0] / batchSize)

    if load_saved_data:
      self.siam1.load_weights(self.location+self.modelName+'_siam1.hdf5')
      self.siam2.load_weights(self.location+self.modelName+'_siam2.hdf5')
      self.Siam.load_weights(self.location+self.modelName+'_siamModelUnited.hdf5')

    # We make the extracted features from classifier non-trainable here.
    if trainFeatureExtractors:
      self.siam1.trainable = True
      self.siam2.trainable = True
    else:
      self.siam1.trainable = False
      self.siam2.trainable = False

    # manually enumerate epochs
    for i in range(Nepochs):
      if i%25 == 0:
        # save the generator model
        self.siam1.save(self.location+self.modelName+'_siam1.hdf5')
        self.siam2.save(self.location+self.modelName+'_siam2.hdf5')
        self.Siam.save(self.location+self.modelName+'_siamModelUnited.hdf5')
      # enumerate batches over the training set
      for j in range(batchPerEp):
        # get randomly selected samples, train and calculate loss
        siamese_inp1,siamese_inp2,labels_siamese = self.__createSiamDataCouples(batchSize, self.data["TrainData"])
        loss, _ = self.Siam.train_on_batch([siamese_inp1,siamese_inp2], labels_siamese)
        
        # calculate validation loss
        siamese_inp1_val,siamese_inp2_val,labels_siamese_val = self.__createSiamDataCouples(batchSize, self.data["ValidationData"])
        outval   = np.squeeze(self.Siam([siamese_inp1_val,siamese_inp2_val]))
        bce      = tf.keras.losses.BinaryCrossentropy(from_logits=True)
        val_loss = bce(labels_siamese_val, outval).numpy()

        print('epoch: %d/%d, %d/%d, loss: %.3f, valloss: %.3f' % (i+1, Nepochs, j+1, batchPerEp, loss, val_loss))
      print("-----------------------------------------------")

    # save the generator model
    self.siam1.save(self.location+self.modelName+'_siam1.hdf5')
    self.siam2.save(self.location+self.modelName+'_siam2.hdf5')
    self.Siam.save(self.location+self.modelName+'_siamModelUnited.hdf5')


In [ ]:
mymodel = model(data,name="Module1", loadSavedModel=True, AppendOptimizedParams=True)

In [ ]:
mymodel.data["TrainData"].keys()

In [ ]:
mymodel.__dict__.keys()

In [ ]:
mymodel.trainClassifier(isTrain=True,epochs=1)

In [ ]:
mymodel.trainSiamese(Nepochs=10, batchSize=512,
                     trainFeatureExtractors= False, 
                     load_saved_data=False)

In [ ]:
mymodel.models["Module1"]

Below is the section to find Cosine Similarity (CS) of the Classes.

The CS is calculated for every class pairs, e.g. 0-0, 0,1, 0,2, etc.

The dictionary "dict_mean_cos" is the dictionary keeping all the values for CS.

In [ ]:
## We define CS4OriginalData function to make the calculation for CS for all class pairs.
## The dictionary "dict_mean_cos" is created inside this function.
def CS4OriginalData(NumberOfDataPickedRandomly,model,dataset1,dataset2):
  siam1            = model
  siam2            = modelDict["siam2"] 
  siamUnited       = modelDict["siamModelUnited"] 
  nOfClass1        = dataset1[2].shape[1]
  dict_mean_cos1   = {str(ii): {"indices":[]} for ii in range(nOfClass1)}
  nOfClass2        = dataset2[2].shape[1]
  dict_mean_cos2   = {str(ii): {"indices":[]} for ii in range(nOfClass2)}
  total_DataNumber1= 0
  total_DataNumber2= 0

  for key in dict_mean_cos1.keys():
    for ii in range(len(dataset1[1])):
      if dataset1[1][ii] == int(key):
        dict_mean_cos1[key]["indices"].append(ii)

  for key in dict_mean_cos2.keys():
    for ii in range(len(dataset2[1])):
      if dataset2[1][ii] == int(key):
        dict_mean_cos2[key]["indices"].append(ii)


  for key in dict_mean_cos1.keys():
    dict_mean_cos1[key]["shape"] = dataset1[1][dict_mean_cos1[key]["indices"]].shape[0]
    print("number of data existing for each class %s: %d for dataset1" % (key,dict_mean_cos1[key]["shape"]))
    total_DataNumber1 = total_DataNumber1 + dict_mean_cos1[key]["shape"]
  
  print("----------------------------------------------------------------------------------------\n")
  
  for key in dict_mean_cos2.keys():
    dict_mean_cos2[key]["shape"] = dataset2[1][dict_mean_cos2[key]["indices"]].shape[0]
    print("number of data existing for each class %s: %d for dataset2" % (key,dict_mean_cos2[key]["shape"]))
    total_DataNumber2 = total_DataNumber2 + dict_mean_cos2[key]["shape"]


  print("----------------------------------------------------------------------------------------\n")
  print("total_data_number for dataset1: %d" % (total_DataNumber1))
  print("dataset1 shape is: ", dataset1[1].shape[0])
  print("----------------------------------------------------------------------------------------")
  print("total_data_number for dataset2: %d" % (total_DataNumber2))
  print("dataset2 shape is: ", dataset2[1].shape[0])
  print("----------------------------------------------------------------------------------------\n")
  for key in dict_mean_cos1.keys():
    try:
      dict_mean_cos1[key]["siam1_Predictions"] = siam1.predict(dataset1[0][dict_mean_cos1[key]["indices"]])
    except:
      continue
  
  for key in dict_mean_cos2.keys():
    try:
      dict_mean_cos2[key]["siam2_Predictions"] = siam2.predict(dataset2[0][dict_mean_cos2[key]["indices"]])
    except:
      continue

  for key1 in dict_mean_cos1.keys():
    len1 = dict_mean_cos1[key1]["shape"]
    for key2 in dict_mean_cos2.keys():
      len2 = dict_mean_cos2[key2]["shape"]   
      
      try:
        list_cs   = []
        list_Siam = []
        lenmin    = np.minimum(len1,len2)
        
        for ii in range(NumberOfDataPickedRandomly):
          ind1, ind2 = np.random.randint(0,lenmin,2)
          ##### HERE IS FOR COSINE SIMILARITY ##########
          cosinesimilarity = 1 - spatial.distance.cosine(dict_mean_cos1[key1]["siam1_Predictions"][ind1], 
                                                         dict_mean_cos2[key2]["siam2_Predictions"][ind2])
          list_cs.append(cosinesimilarity)

          ##### HERE IS FOR SIAMESE SIMILARITY ##########
          siameseResult = siamUnited([dataset1[0][dict_mean_cos1[key1]["indices"][ind1]].reshape(1,1024,2,1),
                                      dataset2[0][dict_mean_cos2[key2]["indices"][ind2]].reshape(1,1024,2,1)])
          list_Siam.append(1 - siameseResult)
        
        string1 = "mean cosine similarity "
        string2 = "mean siamese result"
        print("%s is: %.3f and %s is: %.3f for %s and %s" % (string1,np.mean(list_cs),
                                                             string2,np.mean(list_Siam),
                                                             key1,key2))
      except:
        continue
    
    print("---------------------------------------------------------------------------------------")
  return dict_mean_cos1,dict_mean_cos2

In [ ]:
def OptimumDataPickNumber(list_number=[50]):
  dict_result_OptimizedClassifier1 = {}
  dict_result_OptimizedClassifier2 = {}
  for RandomDataNo in list_number:
    print("RUNNING FOR OptimizedClassifier1")
    _,_ = CS4OriginalData(RandomDataNo,OptimizedClassifier1,dataset,dataset)
    print("RUNNING FOR OptimizedClassifier2")
    _,_ = CS4OriginalData(RandomDataNo,OptimizedClassifier2,dataset,dataset)
  return dict_result_OptimizedClassifier1, dict_result_OptimizedClassifier2
  
_,_ = OptimumDataPickNumber()

In [ ]:
## Here We write the results to pickle.
#unitedResult= [dict_result_OptimizedClassifier1, dict_result_OptimizedClassifier2]
#pickle.dump(unitedResult,open("dict_result_opt.pickle","wb"))

In [ ]:
def OptimumDataPickNumber(list_number=[50]):
  dict_result_OptimizedClassifier1 = {}
  dict_result_OptimizedClassifier2 = {}
  for RandomDataNo in list_number:
    print("RUNNING FOR OptimizedClassifier1")
    _,_ = CS4OriginalData(RandomDataNo,OptimizedClassifier1,dataset,datasetVal)
    print("RUNNING FOR OptimizedClassifier2")
    _,_ = CS4OriginalData(RandomDataNo,OptimizedClassifier2,dataset,datasetVal)
  return dict_result_OptimizedClassifier1, dict_result_OptimizedClassifier2
  
_,_ = OptimumDataPickNumber()

In [ ]:
def OptimumDataPickNumber(list_number=[50]):
  dict_result_OptimizedClassifier1 = {}
  dict_result_OptimizedClassifier2 = {}
  for RandomDataNo in list_number:
    print("RUNNING FOR OptimizedClassifier1")
    _,_ = CS4OriginalData(RandomDataNo,OptimizedClassifier1,dataset,datasetVal_unknown)
    print("RUNNING FOR OptimizedClassifier2")
    _,_ = CS4OriginalData(RandomDataNo,OptimizedClassifier2,dataset,datasetVal_unknown)
  return dict_result_OptimizedClassifier1, dict_result_OptimizedClassifier2
  
_,_ = OptimumDataPickNumber()

## BELOW IS THE PART WHERE WE RUN GENETIC ALGORITM OPTIMIZER

In [ ]:
# Function for MoGenA
def optClassifier(inp):
    print("parameter set is: ",inp)
    filters      = inp[0:4]
    neurons      = inp[4]
    alpha        = inp[5]
    kernelSize   = inp[6]
    Drop         = inp[7]
    learningRate = inp[8]
    classifier   = define_classifier(in_shape=(9,512,1), n_classes=14, 
                                     filters=filters, neurons=neurons,
                                     strides=[(2,1),(2,2),(2,2),(2,2)],
                                     alpha= alpha, kernelSize= kernelSize, Drop= Drop,
                                     learningRate= learningRate)
    history      = classifier.fit(dataset[0], dataset[2], 
                                  batch_size=int(round(inp[9])), 
                                  epochs=int(round(inp[10])), 
                                  validation_data=(datasetVal[0],datasetVal[2]), 
                                  #validation_split = 0.25,
                                  verbose=1, shuffle=True,
                                  callbacks=[checkpoint, reduce_lr])
    minimization_param = -np.max(history.history['val_accuracy'])
    return minimization_param

In [ ]:
# Genetic Algorithm
runOpt = False
if runOpt:
  ga = Mogena.MoGenA(fitness_func         = optClassifier, 
                    number_of_generation = 20,
                    number_of_variables  = 11,
                    population_size      = 30,
                    lower_boundaries     = [8  ,8  ,8  ,8  ,8  ,0.1,3,0.1,0.00001,64  ,20 ],
                    upper_boundaries     = [250,250,250,250,250,0.5,7,0.5,0.001  ,1024,200],
                    use_saved_data       = False)
  result =  {'best_fitness_ever'           : ga.best_fitness_ever,
            'best_individual_decoded_ever' : ga.best_individual_decoded_ever}

Below is the section where we load the synthetically generated data and compare it with the original one using cosine similarity.

In [ ]:
def load_synthetic_data(data_location,classifier=OptimizedClassifier["classifier"]):
  synthetic_data   = pickle.load(open(data_location,"rb"))
  synthetic_data[:,:,0]= scx0.transform(synthetic_data[:,:,0])
  synthetic_data[:,:,1]= scx1.transform(synthetic_data[:,:,1])
  print("Synthetic Data Shape is: ", synthetic_data.shape)
  synthetic_data_result = classifier.predict(synthetic_data)
  listClassifiedSyntheticData = [np.argmax(synthetic_data_result[ii]) for ii in range(len(synthetic_data))]
  print("Generated Classes are: ", listClassifiedSyntheticData)
  return synthetic_data, listClassifiedSyntheticData
synthetic_data, listClassifiedSyntheticData= load_synthetic_data("../mygenerateddata_normal2")

In [ ]:
## CompareCS4SynData is to compare the cosine similarity for the synthetic (syn) data and the original data.
## We compare the synthetic data classes determined by the classifier with the original classes existing in the original dataset
def CompareCS4SynData(synthetic_data,listClassifiedSyntheticData,siam1,siam2,dict_mean_cos):
  res2 = siam2.predict(synthetic_data)
  for key,value in enumerate(listClassifiedSyntheticData):
    res1 = siam1.predict(dataset[0][dict_mean_cos[str(value)]["indices"]])
    list_cs = []
    for ii in range(100):
      ind1 = np.random.randint(0,2500,1)
      cosinesimilarity = 1- spatial.distance.cosine(res1[ind1], res2[key])
      #print(cosinesimilarity)
      list_cs.append(cosinesimilarity)
    string = "mean cosine similarity for class %s and 'newly generated class %s'" % (value,value)
    print("%s is: %.3f" % (string,np.mean(list_cs)))

CompareCS4SynData(synthetic_data,listClassifiedSyntheticData,
                  OptimizedClassifier["siam1"],OptimizedClassifier["siam2"],
                  dict_mean_cos=dict_result_opt["50"])

In [ ]:
plt.scatter(listClassifiedSyntheticData,np.arange(len(listClassifiedSyntheticData)))
plt.show()

In [ ]:
# Postprocess
def postprocess(model,dataset):
    OptimizedClassifier["classifier"].load_weights("./classifier.hdf5")
    results = model.predict(dataset[0])
    list_class = []
    for ii in range(results.shape[0]):
        list_class.append(np.argmax(results[ii]))

    counterTrue  = 0
    counterFalse = 0
    for ii in range(len(list_class)):
        if list_class[ii] == dataset[1][ii]:
            counterTrue = counterTrue + 1
        else:
            counterFalse = counterFalse + 1
    print("True:%d    False:%d" % (counterTrue,counterFalse))
    return list_class,counterTrue,counterFalse

In [ ]:
list_class,counterTrue,counterFalse=postprocess(OptimizedClassifier["classifier"],datasetVal)

## GENERATIVE MODEL (CONDITIONAL GAN)

In [ ]:
class Spectral_Norm(Constraint):
    def __init__(self, power_iters=5):
        self.n_iters = power_iters
        
    def l2_normalize(self, x, eps=1e-12):
        return x / tf.linalg.norm(x + eps)

    def __call__(self, w):
        flattened_w = tf.reshape(w, [w.shape[0], -1])
        u = tf.random.normal([flattened_w.shape[0]])
        v = tf.random.normal([flattened_w.shape[1]])
        for i in range(self.n_iters):
            v = tf.linalg.matvec(tf.transpose(flattened_w), u)
            v = self.l2_normalize(v)
            u = tf.linalg.matvec(flattened_w, v)
            u = self.l2_normalize(u)
        sigma = tf.tensordot(u, tf.linalg.matvec(flattened_w, v), axes=1)
        return w / sigma

    def get_config(self):
        return {'n_iters': self.n_iters}

In [ ]:
# define the standalone discriminator model
def define_discriminator(in_shape=(1024,2,1), n_classes=numberOfClasses, convDim='2D'):
    # label input
    initializer = tf.keras.initializers.GlorotNormal()
    const = Spectral_Norm()
    in_label = Input(shape=(1,16))
    # embedding for categorical input
    li = Embedding(16, 50)(in_label)
    # scale up to image dimensions with linear activation
    n_nodes = in_shape[0] * in_shape[1]
    li = Dense(n_nodes)(li)
    if convDim == '1D':
        # reshape to additional channel
        li = Reshape((in_shape[0], in_shape[1]))(li)
        # image input
        in_image = Input(shape=in_shape[0:2])
        # concat label as a channel
        merge = Concatenate()([in_image, li])
        merge = Reshape((64,64))(merge)
        # downsample
        layer1 = Conv1D(128, (3), strides=(2), padding='same',
                        kernel_initializer = glorot_normal(),
                        kernel_constraint= const)(merge)
        layer1 = LeakyReLU(alpha=0.2)(layer1)

        layer1 = Conv1D(64, (3), strides=(2), padding='same',
                        kernel_initializer = glorot_normal(),
                        kernel_constraint= const)(layer1)
        layer1 = LeakyReLU(alpha=0.2)(layer1)

        layer1 = Conv1D(32, (3), strides=(2), padding='same',
                        kernel_initializer = glorot_normal(),
                        kernel_constraint= const)(layer1)
        layer1 = LeakyReLU(alpha=0.2)(layer1)

    else:
        leakyReluAlpha = 0.2
        ksize = (3,3)
        # reshape to additional channel
        li = Reshape((in_shape[0], in_shape[1], 16))(li)
        # image input
        in_image = Input(shape=in_shape)
        # concat label as a channel
        merge = Concatenate()([in_image, li])
        merge = Reshape((64,32,17))(merge)
        # downsample
        layer1 = Conv2D(128, ksize, strides=(2,1), padding='same',
                        kernel_initializer = initializer,
                        kernel_constraint= const)(merge)
        layer1 = LeakyReLU(alpha=leakyReluAlpha)(layer1)

        layer1 = Conv2D(64, ksize, strides=(2,2), padding='same',
                        kernel_initializer = initializer,
                        kernel_constraint= const)(layer1)
        layer1 = LeakyReLU(alpha=leakyReluAlpha)(layer1)

        layer1 = Conv2D(32, ksize, strides=(2,2), padding='same',
                        kernel_initializer = initializer,
                        kernel_constraint= const)(layer1)
        layer1 = LeakyReLU(alpha=leakyReluAlpha)(layer1)
        
        layer1 = Conv2D(16, ksize, strides=(2,2), padding='same',
                        kernel_initializer = initializer,
                        kernel_constraint= const)(layer1)
        layer1 = LeakyReLU(alpha=leakyReluAlpha)(layer1)

    layer1 = Flatten()(layer1)
    layer1 = Dropout(0.2)(layer1)
    # output
    out_layer = Dense(1, activation='sigmoid')(layer1)
    # define model
    model = Model([in_image, in_label], out_layer)
    # compile model
    opt = RMSprop(lr=0.0001)
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
    model.summary()

    return model

In [ ]:
# size of the latent space
latent_dim = 100

In [ ]:
numberOfClasses

In [ ]:
# define the standalone generator model
def define_generator(latent_dim, n_classes=numberOfClasses, convDim='2D'):
    # label input
    initializer = tf.keras.initializers.GlorotNormal()
    in_label = Input(shape=(1,16))
    # embedding for categorical input
    li = Embedding(16, 50)(in_label)
    # linear multiplication
  
    if convDim== '1D':
        n_nodes = 8 * 16 
        li = Dense(n_nodes)(li)
        # reshape to additional channel
        li = Reshape((64, 2))(li)
        # image generator input
        in_lat = Input(shape=(latent_dim,))
        # foundation for 7x7 image
        n_nodes = 64 * 64
        gen = Dense(n_nodes)(in_lat)
        gen = LeakyReLU(alpha=0.2)(gen)
        gen = Reshape((64, 64))(gen)
        # merge image gen and label input
        merge = Concatenate()([gen, li])
          # upsample to 14x14
        gen = Conv1DTranspose(16, (4), strides=(4), padding='same',kernel_initializer = glorot_normal())(merge)
        gen = LeakyReLU(alpha=0.2)(gen)
        # upsample to 28x28
        gen = Conv1DTranspose(16, (4), strides=(4), padding='same',kernel_initializer = glorot_normal())(gen)
        gen = LeakyReLU(alpha=0.2)(gen)

        gen = Conv1DTranspose(16, (4), strides=(2), padding='same',kernel_initializer = glorot_normal())(gen)
        gen = LeakyReLU(alpha=0.2)(gen)
        # output
        gen = Conv1D(1, (4), activation='tanh', padding='same',kernel_initializer = glorot_normal())(gen)
        out_layer = Reshape((1024, 2))(gen)
    else:
        n_nodes = 4 * 4
        li = Dense(n_nodes)(li)
        # reshape to additional channel
        li = Reshape((16, 4*4, 1))(li)
        # image generator input
        in_lat = Input(shape=(latent_dim,))
        # foundation for 7x7 image
        n_nodes = 4 * 4 * 16
        gen = Dense(n_nodes)(in_lat)
        gen = LeakyReLU(alpha=0.2)(gen)
        gen = Reshape((16, 4*4, 1))(gen)
        # merge image gen and label input
        merge = Concatenate()([gen, li])

        gen = Conv2DTranspose(256, (4,4), strides=(2,2), padding='same',kernel_initializer = initializer)(merge)
        gen = LeakyReLU(alpha=0.2)(gen)

        gen = Conv2DTranspose(128, (4,4), strides=(2,1), padding='same',kernel_initializer = initializer)(gen)
        gen = LeakyReLU(alpha=0.2)(gen)

        gen = Conv2DTranspose(128, (4,4), strides=(1,1), padding='same',kernel_initializer = initializer)(gen)
        gen = LeakyReLU(alpha=0.2)(gen)

        gen = Conv2DTranspose(64, (4,4), strides=(1,1), padding='same',kernel_initializer = initializer)(gen)
        gen = LeakyReLU(alpha=0.2)(gen)
    
        gen = Conv2D(1, (3,3), strides=(1,1), activation='tanh', padding='same',
                     kernel_initializer = initializer)(gen)
        gen = Dropout(0.2)(gen)
    out_layer = Reshape((1024, 2, 1))(gen)
    # define model
    model = Model([in_lat, in_label], out_layer)
    model.summary()
    return model

In [ ]:
# Inception Score for measuring the similarity between the generated signals and the real ones, using KL Divergence
def InceptionScore(nelem=100):
    for tt in range(1):
        RealImageList = []
        latent_points, labels = generate_latent_points(latent_dim=100, n_samples=30)
        for elem in range(labels.shape[0]):
            nolabel = np.argmax(labels[elem])
            for ii in range(1000):
                if dataset[1][ii] == nolabel:
                    indice = ii
        
            RealImageList.append(dataset[0][indice])

        RealImageList= np.array(RealImageList)
        X1  = g_model.predict([latent_points, labels])
        kl  = KLDivergence()
        bce = BinaryCrossentropy(from_logits=True)

    return kl(RealImageList,X1).numpy()

In [ ]:
# define the combined generator and discriminator model, for updating the generator
def define_gan(g_model, d_model):
    # make weights in the discriminator not trainable
    #d_model.trainable = False

    in_label = Input(shape=(1,16))
    in_lat = Input(shape=(latent_dim,))

    gen_out = g_model([in_lat,in_label])

    gan_out = d_model([gen_out,in_label])

    # get noise and label inputs from generator model
    #gen_noise, gen_label = g_model.input
    # get image output from the generator model
    #gen_output = g_model.output
    # connect image output and label input from generator as inputs to discriminator
    #gan_output = d_model([gen_output, gen_label])

    # define gan model as taking noise and label and outputting a classification
    model = Model([in_lat,in_label], gan_out)
    # compile model
    opt = RMSprop(lr=0.0003)
    model.compile(loss='binary_crossentropy', optimizer=opt)
    #model.compile(loss=Ganloss, optimizer=opt)
    model.summary()

    return model

In [ ]:
dataset[2].shape

In [ ]:
# select real samples
def generate_real_samples(dataset, n_samples):
	# split into images and labels
	images, labels, onehotenc = dataset
	# choose random instances
	ix = randint(0, images.shape[0], n_samples)
	# select images and labels
	X, labels, onehotenc = images[ix], labels[ix], onehotenc[ix]
	onehotenc = onehotenc.reshape((n_samples,1,16))
	# generate class labels
	y = np.ones((n_samples, 1))
	return [X, onehotenc], [y, onehotenc]

In [ ]:
# generate points in latent space as input for the generator
def generate_latent_points(latent_dim, n_samples, n_classes=numberOfClasses):
	# generate points in the latent space
	x_input = randn(latent_dim * n_samples)
	# reshape into a batch of inputs for the network
	z_input = x_input.reshape(n_samples, latent_dim)
	# generate labels
	labels = randint(0, n_classes, n_samples)
	dumbarray = np.zeros((n_samples,1,16))
	for ii in range(n_samples):
		indice = labels[ii]
		dumbarray[ii,0,indice] = 1
	return [z_input, dumbarray]

In [ ]:
# use the generator to generate n fake examples, with class labels
def generate_fake_samples(generator, z_input, labels_input, n_samples):
	# predict outputs
	images = generator.predict([z_input, labels_input])
	# create class labels
	y = np.zeros((n_samples, 1))
	return [images, labels_input], y

In [ ]:
# create the discriminator
d_model = define_discriminator(convDim='2D')
tf.keras.utils.plot_model(d_model, show_shapes=True)

In [ ]:
# create the generator
g_model = define_generator(latent_dim,convDim='2D')
tf.keras.utils.plot_model(g_model, show_shapes=True)

In [ ]:
# create the gan
gan_model = define_gan(g_model, d_model)
tf.keras.utils.plot_model(gan_model,show_shapes=True)

In [ ]:
# train the generator and discriminator
def train(g_model, d_model, gan_model, dataset, latent_dim, 
          n_epochs=100, n_batch=128, load_saved_data=True, 
          dclip_value=0.1, gclip_value=0.1):
    
    if load_saved_data:
        g_model.load_weights('cgan_generator.hdf5')
        d_model.load_weights('cgan_discriminator.hdf5')
  
    bat_per_epo = int(dataset[0].shape[0] / n_batch)
    half_batch = int(n_batch / 2)
    # manually enumerate epochs
    for i in range(n_epochs):
        if i%10 == 0:
          # save the generator model
            g_model.save('cgan_generator.hdf5')
            d_model.save('cgan_discriminator.hdf5')
        # enumerate batches over the training set
        for j in range(bat_per_epo):
            list_z_input = []
            list_labels_input =[]
            d_model.trainable = True
            for ii in range(5):
                # get randomly selected 'real' samples
                [X_real, labels_real], [y_real, onehotenc] = generate_real_samples(dataset, n_batch)
                # update discriminator model weights
                d_loss1, _ = d_model.train_on_batch([X_real, labels_real], y_real)
                # generate 'fake' examples
                # prepare points in latent space as input for the generator
                [z_input, labels_input]  = generate_latent_points(latent_dim, n_batch)
                [X_fake, labels], y_fake = generate_fake_samples(g_model, z_input, labels_input, n_batch)
                # update discriminator model weights
                d_loss2, _ = d_model.train_on_batch([X_fake, labels], y_fake)

                list_z_input.append(z_input)
                list_labels_input.append(labels_input)

                for l in d_model.layers:
                    weights = l.get_weights()
                    weights = [np.clip(w, -dclip_value, dclip_value) for w in weights]
                    l.set_weights(weights)

            zinput = np.ndarray((5*n_batch, latent_dim))
            labelsinput = np.ndarray((5*n_batch,1,16))
            cntr = 0
            for ii in range(5):
                for jj in range(n_batch):
                    zinput[cntr,:] = list_z_input[ii][jj,:]
                    labelsinput[ii*n_batch:(ii+1)*n_batch,:] = list_labels_input[ii][:]
                    cntr = cntr + 1

            # create inverted labels for the fake samples
            y_gan = np.ones((5*n_batch, 1))
            # update the generator via the discriminator's error
            d_model.trainable = False
            #g_loss = gan_model.train_on_batch([z_input, labels_input], y_gan)
            g_loss = gan_model.train_on_batch([zinput, labelsinput], y_gan)

            for l in g_model.layers:
                weights = l.get_weights()
                weights = [np.clip(w, -gclip_value, gclip_value) for w in weights]
                l.set_weights(weights)
            # summarize loss on this batch
            try:
                inceptionScore = InceptionScore(nelem=n_batch)
            except:
                continue
            print('epoch: %d/%d, %d/%d, dloss_real: %.3f, dloss_fake: %.3f, gloss: %.3f, IS: %s' %\
                  (i+1, n_epochs, j+1, bat_per_epo, d_loss1, d_loss2, g_loss, inceptionScore))

        ix = np.random.randint(0,half_batch)
        #print('generated label : ', labels)
        #print("***************************")
        #print('real label      : ', labels_real)
        X_real_abs = X_real[ix,:,0] + 1j*X_real[ix,:,1]
        X_fake_abs = X_fake[ix,:,0] + 1j*X_fake[ix,:,1]

        print('\n')
        plt.figure(figsize=(15,6))
        plt.plot(np.abs(X_real_abs))
        plt.plot(np.abs(X_fake_abs))
        plt.show()
        print('\n')

    # save the generator model
    g_model.save('cgan_generator.hdf5')
    d_model.save('cgan_discriminator.hdf5')

In [ ]:
# train model
train(g_model, d_model, gan_model, dataset, latent_dim, 
      n_epochs=100, n_batch=256, load_saved_data=True, 
      dclip_value=0.1, gclip_value=0.5)    

In [ ]:
g_model.load_weights('cgan_generator.hdf5')
d_model.load_weights('cgan_discriminator.hdf5')

In [ ]:
# example of loading the generator model and generating images
def generateNewSamples():
	n_samples_perclass = 150
	n_classes          = numberOfClasses
	
	# generate points in latent space as input for the generator
	def generate_latent_points(latent_dim, n_samples_perclass, n_classes=n_classes):
		numberOfsamples = n_samples_perclass*n_classes
		# generate points in the latent space
		x_input = randn(latent_dim * numberOfsamples)
		# reshape into a batch of inputs for the network
		z_input = x_input.reshape(numberOfsamples, latent_dim)
		# generate labels
		onehot = np.zeros((n_classes*n_samples_perclass,1,16))
		labels = np.ndarray((n_classes*n_samples_perclass,)) 
		for ii in range(n_classes):
			for jj in range(n_samples_perclass):
				onehot[ii*n_samples_perclass+jj,0,ii] = 1
				labels[ii*n_samples_perclass+jj] = ii
		return [z_input, labels, onehot]
	

	# load model
	model = load_model('cgan_generator.hdf5')
	# generate images
	latent_points, labels, onehot = generate_latent_points(latent_dim=100, n_samples_perclass=n_samples_perclass)

	# generate images
	X1  = model.predict([latent_points, onehot])
	#X1  = inv_scaling(X1,max,min)
	mygenerateddata = []
	mygenerateddata.append(X1)
	mygenerateddata.append(labels)
	mygenerateddata.append(onehot)

	#print(X1.shape)
	for ii in range(n_samples_perclass*n_classes):
		print("label: ",labels[ii])
		plt.plot(X1[ii,:,0,0])
		plt.plot(X1[ii,:,1,0])
		plt.show()
		print("******************")

	return mygenerateddata

In [ ]:
mygenerateddata = generateNewSamples()

In [ ]:
mygenerateddata_inv = mygenerateddata
mygenerateddata_inv[0][:,:,0,0] = scx0.inverse_transform(mygenerateddata[0][:,:,0,0])
mygenerateddata_inv[0][:,:,1,0] = scx1.inverse_transform(mygenerateddata[0][:,:,1,0])

In [ ]:
import pickle
pickle.dump(mygenerateddata_inv,open("generated_inv01.pickle","wb"))
mygenerateddata_inv[0].shape

In [ ]:
plt.plot(mygenerateddata_inv[0][0:100,:,1,0])
plt.show()

In [ ]:
def mergeDatasets():
    print("generateddata shape: ",mygenerateddata[0].shape)
    print("original data shape before merging: ",dataset[0].shape)

    mygenerateddata[2] = mygenerateddata[2].squeeze()

    list_data_base = [dataset,mygenerateddata]
    total_data = len(dataset[0]) + len(mygenerateddata[0])

    ## Merging the Generated data set to the existing one
    newdataset = np.ndarray((total_data,dataset[0].shape[1],dataset[0].shape[2],dataset[0].shape[3]))
    newlabels  = np.ndarray((total_data,))
    newonehot  = np.ndarray((total_data,16))

    newdataset[0:len(dataset[0])] = dataset[0]
    newdataset[len(dataset[0]):] = mygenerateddata[0]

    newlabels[0:len(dataset[1])] = dataset[1]
    newlabels[len(dataset[1]):] = mygenerateddata[1]

    newonehot[0:len(dataset[2])] = dataset[2]
    newonehot[len(dataset[2]):] = mygenerateddata[2]

    print("dataset shape after merging: ",newdataset.shape)
    datasetNew = [newdataset,newlabels,newonehot]
    return datasetNew


In [ ]:
MergedDataset = mergeDatasets()

In [ ]:
list_classLabels,counterTrue,counterFalse=postprocess(classifier,dataset)
list_classLabelsGen,counterTrue,counterFalse=postprocess(classifier,mygenerateddata)
list_classLabelsMerged,counterTrue,counterFalse=postprocess(classifier,MergedDataset)


In [ ]:
print("Generated Classes after Conditional GAN training")
plt.scatter(np.arange(len(list_classLabelsGen)),mygenerateddata[1])
plt.show()

In [ ]:
print("Generated Classes Determined by the Classifier Trained with Original Dataset")
plt.scatter(np.arange(len(list_classLabelsGen)),list_classLabelsGen)
plt.show()

Sonuçlar incelendiğinde:
    - Her sınıftan eşit sayıda sınıf oluşturması gerekirken bunu yapmadığı,
    - Conditional GAN'ın 5, 7, ve 8. sınıflardan yeni sınıf oluşturmada zorlandığı,
    - oluşturulan sınıfların classifier tarafından sınıflandırıldığında, farklı label'lı sınıf olarak gördüğü
anlaşılmakta.

Ancak, yine de kontrolsüz de olsa, birden fazla sınıf için yeni veri üretebilmiş gibiyiz. Bunu test etmek için knn vb yöntemleri de kullanarak sınıflandırma yapalım.